<a href="https://colab.research.google.com/github/ronykroy/DNN-NLP-and-other-stuff/blob/master/H20_multiClass_newsGroups_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install default-jre
!java -version
# install Java.. because.. there is nothing in this colab VM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [2]:
! pip install h2o
# H2O too to be installed SMH

     |████████████████████████████████| 123.6MB 79kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.8-py2.py3-none-any.whl size=123708939 sha256=82097c8bf6b2837e612e90652b4f289d2a07cd6d2d9dd55cb80ead4438b6ce48
  Stored in directory: /root/.cache/pip/wheels/f7/4c/2f/367de2996c0c2621a9e2e19624be0ea0e4b766df3cd597d916
Successfully built h2o


In [0]:
import h2o

In [4]:
h2o.init(max_mem_size="4G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprkmy7yf9
  JVM stdout: /tmp/tmprkmy7yf9/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprkmy7yf9/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_unknownUser_40fmn5
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [5]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

INFO:sklearn.datasets.twenty_newsgroups:Downloading 20news dataset. This may take a few minutes.
INFO:sklearn.datasets.twenty_newsgroups:Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [0]:
df["Train"] = np.random.rand(len(df)) < 0.8 # get a col full of train boolean 

In [9]:
df.Train.value_counts()

True     9051
False    2263
Name: Train, dtype: int64

In [10]:
reviews = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
train = reviews[reviews["Train"] == "True"]
test = reviews[reviews["Train"] == "False"]

In [12]:
train.head(2)

label,text,Train
17,"Well i'm not sure about the story nad it did seem biased. What I disagree with is your statement that the U.S. Media is out to ruin Israels reputation. That is rediculous. The U.S. media is the most pro-israeli media in the world. Having lived in Europe I realize that incidences such as the one described in the letter have occured. The U.S. media as a whole seem to try to ignore them. The U.S. is subsidizing Israels existance and the Europeans are not (at least not to the same degree). So I think that might be a reason they report more clearly on the atrocities. What is a shame is that in Austria, daily reports of the inhuman acts commited by Israeli soldiers and the blessing received from the Government makes some of the Holocaust guilt go away. After all, look how the Jews are treating other races when they got power. It is unfortunate.",True
0,"Yeah, do you expect people to read the FAQ, etc. and actually accept hard atheism? No, you need a little leap of faith, Jimmy. Your logic runs out of steam! Jim, Sorry I can't pity you, Jim. And I'm sorry that you have these feelings of denial about the faith you need to get by. Oh well, just pretend that it will all end happily ever after anyway. Maybe if you start a new newsgroup, alt.atheist.hard, you won't be bummin' so much? Bye-Bye, Big Jim. Don't forget your Flintstone's Chewables! :) -- Bake Timmons, III",True


In [0]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [0]:
# TODO: review how effective this is in removing junk \r \n etc
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [15]:
%%time
# Break reviews into sequence of words
words = tokenize(reviews["text"]) # not sure if there is some sort of leakage here...

CPU times: user 3.6 s, sys: 58.3 ms, total: 3.66 s
Wall time: 19.3 s


In [16]:
words.head()

C1
well
sure
story
nad
seem
biased
disagree
statement
media
ruin


In [17]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

# This takes time to run - left commented out
w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words) # train on the words thing we just got

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [18]:
# Sanity check - find synonyms for the word 'president'
w2v_model.find_synonyms("president", count = 5) # not bad actually :)

OrderedDict([('yeltsin', 0.8303556442260742),
             ('mubarak', 0.8219935297966003),
             ('stephanopoulos', 0.7914628982543945),
             ('miyazawa', 0.7749645709991455),
             ('briefing', 0.7329180240631104)])

In [0]:
# Calculate a vector for each review
review_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [20]:
review_vecs

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.133946,0.157323,-0.0249578,-0.182409,0.0959795,0.156111,0.021005,0.0276131,0.105582,-0.113926,-0.0112111,-0.0542047,0.0816997,-0.0310789,-0.278647,-0.106389,-0.0567617,-0.0491182,0.0120456,-0.0213543,-0.000997308,0.0543348,-0.0479194,-0.0600538,-0.0759979,0.0706979,0.0100861,-0.0608743,0.0256549,-0.0470876,-0.154995,0.213828,-0.0322199,-0.183268,-0.0596899,0.187403,0.0573363,0.00985568,-0.137973,-0.219322,-0.0808192,0.0549824,0.0716381,-0.156614,0.00556052,0.0605803,0.0298139,-0.0842324,-0.0205442,-0.154897,0.0585786,-0.038809,0.0250581,0.0668402,-0.0241437,0.187213,0.0551689,0.0444126,-0.102073,0.125266,-0.138356,-0.044337,0.129926,-0.0525584,0.00544784,0.20003,-0.0718302,-0.090649,-0.0370178,0.0101713,0.0482812,-0.0573931,0.136591,-0.0338863,-0.057611,-0.123168,0.15538,0.143941,-0.0151191,-0.0820133,0.0513678,0.0598795,0.0521264,0.0734729,0.0355231,0.124026,0.0131578,-0.0250693,-0.03276,0.0370808,0.044777,-0.114106,-0.0806621,0.0273033,0.0608677,0.186664,0.17342,0.0408443,0.0889113,0.16076
0.218499,0.0899447,-0.0735187,-0.0952984,0.0142813,0.020982,0.0477001,-0.0553651,0.0743127,0.000960571,0.0559127,-0.0472865,0.0704115,0.000103468,-0.226459,-0.110831,-0.0386106,-0.05838,-0.0670829,-0.0186162,-0.0159057,0.0953788,-0.0469174,-0.016423,-0.0531325,0.0892219,0.0539667,-0.107982,0.155148,0.0280835,-0.0279609,0.0345187,-0.0310675,-0.04546,-0.0224368,0.124488,0.14517,-0.0633337,-0.197232,-0.0315936,-0.000123122,-0.0136036,0.0370295,-0.116221,0.0905541,0.0119156,0.0167236,-0.172159,-0.022037,-0.122361,0.0370234,-0.0394331,-0.0245718,-0.0500245,0.0215894,0.104113,-0.0310412,-0.0243576,-0.246465,0.0182768,-0.166982,-0.0395235,-0.0781111,-0.0911497,-0.00198084,0.28996,-0.0343353,-0.0938801,-0.0197342,0.0704103,-0.00169383,0.0126904,0.117714,-0.0367284,-0.11733,-0.0768908,0.0642986,0.0697737,-0.0297251,-0.0685437,0.0475003,0.0193253,0.102115,-0.00498718,-0.0180369,0.158683,0.13858,0.0601666,-0.036972,-0.0396414,-0.160107,-0.0245319,-0.0476622,0.0866378,0.0568917,0.246424,0.224963,0.0569448,0.0877509,0.0943768
0.179759,0.187554,0.00970632,-0.190007,0.0772827,0.110808,0.0102224,0.00131369,0.0720183,-0.0996448,-0.0390486,-0.0452679,0.0536141,0.0173678,-0.21197,-0.151909,-0.0259862,-0.0541615,0.00566619,-0.0480715,0.0252596,0.0156971,0.0379359,-0.00868671,-0.0377183,0.089569,-0.0127786,-0.0351713,0.0528945,-0.0497415,-0.119408,0.148709,-0.00379205,-0.12861,-0.0487258,0.171725,0.113342,-0.00726895,-0.141939,-0.224271,-0.0591935,0.180895,-0.0363565,-0.141095,0.0161304,0.0937432,0.0556235,-0.115029,-0.0615909,-0.153856,0.0847529,-0.044601,0.0745332,0.117221,-0.020912,0.14334,-0.0113995,0.0183666,-0.166496,0.116435,-0.146971,0.0448003,0.100269,-0.0878179,-0.00368638,0.206793,-0.0808142,-0.112168,-0.00267418,0.0354657,0.0483186,-0.0104061,0.0816039,0.0268207,-0.0487451,-0.110701,0.0691297,0.131534,0.0243836,-0.0853761,0.021688,0.0687536,-0.0139939,0.037345,-0.00353219,0.145932,0.0181816,0.0579587,-0.0875608,-0.0489963,0.0206857,-0.108844,-0.076198,0.108642,0.0766888,0.213767,0.19163,0.0264709,0.118265,0.193578
0.251815,0.170776,-0.0325758,-0.126982,-0.019516,0.0608466,0.0539654,0.0913558,0.102451,-0.00425833,0.0528539,-0.00577321,0.0366463,-0.00162944,-0.169667,-0.195597,-0.0535023,-0.0651195,-0.065354,0.0449501,0.0541024,0.0813348,-0.00131745,0.125368,-0.0166005,0.0979159,-0.0251909,-0.0995574,0.0702081,-0.0447503,-0.112218,0.081527,0.117081,-0.0242586,-0.0653195,0.110109,0.0993826,0.00093166,-0.133637,-0.16369,-0.0222842,-0.00061366,-0.136848,-0.117513,0.118512,-0.0216018,0.0658273,-0.13155,-0.00272537,-0.116839,0.0694264,-0.00427334,0.0780613,0.0841225,0.

In [0]:
# Add aggregated word embeddings 
ext_reviews = reviews.cbind(review_vecs)
# rename may be... all 'reviews' to 'news'
# ext becaus ewe are extending the reviews df with the vectors

In [0]:
ext_reviews["label_f"]  = ext_reviews["label"] .asfactor()

In [0]:
# Splitting to test train.. along with the review vecs is difficult without the 'Train' boolean column
# hence redoing it from that stage..
# so that we'll be able to do this following cell

In [0]:
ext_train = ext_reviews[ext_reviews["Train"] == "True"]
ext_test = ext_reviews[ext_reviews["Train"] == "True"]

In [0]:
#review_vecs.names
# had a 100 cols
# may be more would give a better model ..?

In [0]:
predictors =  review_vecs.names

In [0]:
response = 'label_f'

In [0]:
from h2o.automl import H2OAutoML

In [0]:
# Identify predictors and response
x = ext_train.columns
y = "label_f"
x.remove(y)

# For binary classification, response should be a factor
#train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()
# Already done

In [0]:
# drop even the 'label' and 'train' columns..? from the predictors list..?

In [30]:
%%time
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=ext_train)


AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 16 s, sys: 804 ms, total: 16.8 s
Wall time: 1h 4min 42s


In [31]:

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20191024_044349,0,0.0115687,0.0185751,0.000345036
GBM_5_AutoML_20191024_044349,0,0.00400766,0.0105039,0.000110331
GBM_2_AutoML_20191024_044349,0.00103108,0.00779038,0.04005,0.001604
GBM_1_AutoML_20191024_044349,0.00103108,0.0101256,0.0392171,0.00153798
GBM_3_AutoML_20191024_044349,0.00167366,0.0157199,0.0496454,0.00246466
GBM_grid_1_AutoML_20191024_044349_model_1,0.00175835,1.71118,0.814336,0.663143
GBM_4_AutoML_20191024_044349,0.00186791,0.0204565,0.0550861,0.00303448
XRT_1_AutoML_20191024_044349,0.0142495,0.526169,0.420463,0.176789
DRF_1_AutoML_20191024_044349,0.0254083,0.590468,0.450175,0.202657
GBM_grid_1_AutoML_20191024_044349_model_2,0.0255786,0.13753,0.174937,0.0306031


In [34]:
#aml.leader.confusion_matrix()
aml.leader.cross_validation_metrics_summary

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20191024_044349

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.00011398483929601745
RMSE: 0.010676368263413241

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.00034503552140033165
RMSE: 0.018575131800348865


<bound method ModelBase.cross_validation_metrics_summary of >

In [35]:
aml.leader.confusion_matrix(ext_test) # wow !!!!!!!!


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,115,460,Error,Rate
0,388.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 388
1,0.0,457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 457
2,0.0,0.0,461.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 461
3,0.0,0.0,0.0,477.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 477
4,0.0,0.0,0.0,0.0,473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 473
5,0.0,0.0,0.0,0.0,0.0,464.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 464
6,0.0,0.0,0.0,0.0,0.0,0.0,474.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 474
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,472.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 472
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,481.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 481
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,468.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 468



See the whole table with table.as_data_frame()
